# Exercícios de Análise com SQL Avançado
## U.S. EPA Food Commodity Intake Database (FCID)
### [https://fcid.foodrisk.org/](https://fcid.foodrisk.org/)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

# Importando Tabelas do FCID

In [2]:
DROP TABLE IF EXISTS Crop_Group;
DROP TABLE IF EXISTS FCID_Description;
DROP TABLE IF EXISTS Recipes;
DROP TABLE IF EXISTS Intake;

CREATE TABLE Crop_Group (
  CGN VARCHAR(2),
  CGL VARCHAR(6),
  Crop_Group_Description VARCHAR(80),
  PRIMARY KEY (CGL)
) AS SELECT
  CGN, CGL, Crop_Group_Description
FROM CSVREAD('../../../data/food-intake/basics/FCID_Cropgroup_Description.csv');

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code),
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../../../data/food-intake/basics/FCID_Code_Description.csv');

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../../../data/food-intake/recipes/Recipes_WWEIA_FCID_0510.csv');

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../../../data/food-intake/consumption/Commodity_CSFFM_Intake_0510-cropped.csv');

# Visualizando as Tabelas

## Grupos e Subgrupos de Cultura (Agrícolas)

Cada linha é um grupo ou subgrupo. Quando o valor de `CGN` é igual a `CGL`, trata-se da descrição de um grupo, senão de um subgrupo.

Campo | Descrição
--- | ---
CGN | código do grupo de cultura (agrícola)
CGL | código do subgrupo de cultura (agrícola)
Crop_Group_Description | descrição do grupo ou subgrupo de cultura

In [3]:
SELECT * FROM Crop_Group LIMIT 10;

## Ingredientes (Produtos Alimentares de Base)

Cada tupla é um produto alimentar de base usado em receitas.

| Campo | Descrição | Chave para a Tabela |
| --- | --- | --- |
| CGN | código do grupo de cultura (agrícola) | associado à tabela de Grupos e Subgrupos de Cultura |
| CG_Subgroup | código do subgrupo de cultura (agrícola) | associado à tabela de Grupos e Subgrupos de Cultura |
| FCID_Code | código do ingrediente | |
| FCID_Desc | descrição do ingrediente | |

In [4]:
SELECT * FROM FCID_Description LIMIT 10;

## Receitas - Composição

Cada tupla é um item que é parte de uma receita. Cada receita - todas as tuplas com o mesmo `Food_code`.

| Campo | Descrição | Chave para a Tabela |
| --- | --- | --- |
| Food_Code | código da receita | Alimentos feitos por Receitas |
| Mod_Code | código de receita modificada (0 = sem modificação) | |
| Ingredient_Num | número do ingrediente dentro da receita | |
| FCID_Code | código do ingrediente | Ingredientes (Produtos Alimentares) |
| Cooked_Status | estado e cozimento original do ingrediente | Estado de Cozimento do Produto |
| Food_Form | forma alimentar | Forma do Produto Alimentar |
| Cooking_Method | método de cozimento do ingrediente na receita | Método de Cozimento do Produto na Receita |
| Commodity_Weight | participação (em percentual) do ingrediente no peso da receita | |

In [5]:
SELECT * FROM Recipes LIMIT 10;

## Consumo

Cada tupla é um produto alimentar (ingrediente) ingerido por uma pessoa em um dia. Todas as tuplas com mesmo `SEQN` são ingestões da mesma pessoa no dia 1 ou 2 (`DAYCODE`).

| Campo | Descrição | Chave para a Tabela ou Valor |
| --- | --- | --- |
| SEQN  | id do entrevistado | |
| DRABF | indica se o entrevistado (bebê) foi amamentado em qualquer um dos dois dias de retorno | 1 = sim; 2 = não |
| DAYCODE | Dia do diário alimentar. | 1 = dia 1; 2= dia 2 |
| FCID_Code | código do produto alimentar (usado como ingrediente) | Ingredientes (Produtos Alimentares) |
| Cooked_Status | estado e cozimento original do ingrediente | Estado de Cozimento do Produto |
| Food_Form | forma alimentar | Forma do Produto Alimentar |
| Cooking_Method | método de cozimento do ingrediente na receita | Método de Cozimento do Produto na Receita |
| Intake | ingestão. em gramas | |
| Intake_BW | ingestão. em gramas por quilograma de peso corporal (g/kg pc) | |

In [6]:
SELECT * FROM Intake LIMIT 10;

# Métricas

Considere que a tabela Intake registra alimentos consumidos por 1.489 pessoas. Considere as seguintes métricas para um alimento:

| Métrica | Descrição |
| --- | --- |
| Popularidade | número de pessoas (dentre as 1.489) que consumiram o alimento |
| Intake_Sum | total consumido do alimento pelas 1.489 pessoas em gramas |
| Intake_AVG | média de consumo do alimento em gramas |
| Intake_AVG_BW | média de consumo do alimento x peso da pessoa |
| Recipes | número de receitas (dentre as 7.154 receitas) que têm o produto como ingrediente |

## 1)  Construa uma View que apresente essas métricas por produto

* Veja exemplo em: `/data/food-intake/computed/commodity-profile.csv`
* Importante: esta tabela foi feita com um número maior de registros, portanto os valores não serão iguais aos seus

## 2) Como você analisaria a correlação entre as métricas?

* Por exemplo, produtos mais populares são mais consumidos (em número de pessoas ou em quantidade)?
* Proponha uma ou mais queries para fazer esta análise

## 3) Podemos criar grupos de consumidores conforme um perfil?
* por exemplo, consumidores podem ser agrupados por alimentos que comem predominantemente?
* como você associaria grupos a classes?

## 4) Que métricas podem ser analisadas para a comparação de perfis?
* escreva uma query SQL que calcule pelo menos uma métrica comparativa